# **Get datasets**

In [2]:
import tensorflow_datasets as tfds
import numpy as np
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

# Need numpy arrays instead of straight arrays for training:  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

# **Tokenizer and pad sequences**

In [3]:
# Hyperparameters on top for easy modification.
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [4]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3])) # Processed sentence output
print(training_sentences[3]) # Original sentence

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? b'this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received '
b'This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.'


# **Define Model**

In [6]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') # Only 2 output classes
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [7]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 5s 6ms/step - loss: 0.5013 - accuracy: 0.7398 - val_loss: 0.3519 - val_accuracy: 0.8465
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2462 - accuracy: 0.9045 - val_loss: 0.3640 - val_accuracy: 0.8428
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0989 - accuracy: 0.9740 - val_loss: 0.4440 - val_accuracy: 0.8278
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0261 - accuracy: 0.9967 - val_loss: 0.5431 - val_accuracy: 0.8250
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0089 - accuracy: 0.9992 - val_loss: 0.5674 - val_accuracy: 0.8299
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0045 - accuracy: 0.9994 - val_loss: 0.6228 - val_accuracy: 0.8269
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0021 - accuracy: 0.9998 - val_loss: 0.6832 - val_accuracy: 0.8263
Epoch 